<a href="https://colab.research.google.com/github/esma6/APP-main/blob/main/DR-GaussianBlur-CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [15]:
%cd gdrive/MyDrive/Datasets/

/content/gdrive/MyDrive/Datasets


In [16]:
# Gerekli kütüphaneler yüklenir
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical

In [17]:
# Veri kümesi yüklenir
train_dir = "GaussianBlur"
train_csv = "train.csv"
df_train = pd.read_csv(train_csv)


In [18]:
# Görüntüler ve etiketler yüklenir
x = []
y = []
for i, row in df_train.iterrows():
    img_path = os.path.join(train_dir, row["id_code"] + ".png")
    img = cv2.imread(img_path)
    img = cv2.resize(img, (256, 256))
    x.append(img)
    y.append(row["diagnosis"])
x = np.array(x)
y = np.array(y)

In [20]:
# Veri seti karıştırılır ve eğitim/test setleri oluşturulur
x, y = shuffle(x, y, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [21]:
# Etiketler kategorik hale getirilir
y_train = to_categorical(y_train, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

In [22]:
# Model tanımlanır
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [23]:
# Model derlenir
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
# Model eğitilir
model.fit(x_train[0:3000], y_train[0:3000], batch_size=16)

184/184 [==============================] - 572s 3s/step - loss: 11.8171 - accuracy: 0.6763


In [25]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.9616519212722778
Test accuracy: 0.6793997287750244
